# ORB-SLAM3 VSLAM Tutorial

This tutorial goes through examples of ORB-SLAM3 Python binding.

In [ ]:
# Install dependencies
%pip install opencv-python matplotlib jupyter_bokeh ipywidgets tqdm scipy torchdata

In [1]:
import cv2
import os
import re
import sys
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
from tqdm.notebook import tqdm
from IPython.display import HTML
from typing import Iterator, Tuple, Optional
from scipy.spatial.transform import Rotation

sys.path.append(os.path.join(os.getcwd(), "..", "lib"))
import orb_slam3

matplotlib.rcParams['animation.embed_limit'] = 40 # MB

This tutorial uses [Bokeh](https://bokeh.org/) that visualizes interactive graphs. First, initialize bokeh and then define a helper function that displays an embeded video in the notebook.

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.layouts import row

TOOLTIPS = [
  ("X","@x"),
  ("Y","@y")
]

output_notebook()    

# Helper that displays an inline video
def display_video(frames, framerate=30, repeat=False):
    h, w = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')
    fig, ax = plt.subplots(1, 1, figsize=(w / dpi, h / dpi), dpi=dpi)
    matplotlib.use(orig_backend)
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0], cmap='gray', vmin=0, vmax=255)
    def update(frame):
        im.set_data(frame)
        return [im]
    interval = 1000 / framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames, 
                                   interval=interval, blit=True, repeat=repeat)
    return HTML(anim.to_html5_video())

def display_video2(dp, size):
    ts, frames = list(), list()
    for t, l, r, _ in tqdm(list(dp)):
        ts.append(t)
        frames.append(cv2.hconcat([cv2.resize(l,dsize=size), cv2.resize(r,dsize=size)]))
    fps = len(frames) / ((ts[-1]-ts[0]) / 1e9)
    print("fps: %d" % math.floor(fps))
    return display_video(frames, fps)


Loading BokehJS ...

## Dataset Loaders

In [3]:
# Data Pipe
from operator import itemgetter
from torchdata.datapipes.iter import IterableWrapper

CAM0_IMG_PAT = re.compile(r"cam0/data/[0-9]+[.]png")
CAM1_IMG_PAT = re.compile(r"cam1/data/[0-9]+[.]png")
IMU0_CSV_PAT = re.compile(r"imu0/data[.]csv")

def collate_cam(data):
    path, im = data
    t = int(os.path.basename(path)[0:-4])
    im = cv2.imdecode(np.frombuffer(im.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
    return t, im

def classifier(data):
    path, _ = data
    if bool(IMU0_CSV_PAT.search(path)):
        return 0
    if bool(CAM0_IMG_PAT.search(path)):
        return 1
    elif bool(CAM1_IMG_PAT.search(path)):
        return 2
    else:
        return None

class Compounder:
    def __init__(self, imu_dp):
        self._imu = np.asarray(list(imu_dp))
        self._t_last = 0

    def __call__(self, cam0, cam1):
        t = cam0[0]
        if len(self._imu.shape) == 1:
            imu = np.empty(0)
        else:
            imu = self._imu[self._imu[:,0] > self._t_last]
            imu = imu[imu[:,0] <= t]
        self._t_last = t
        return t, cam0[1], cam1[1], imu

def VIDataPipe(uri):    
    """Vision Inertial Data Pipeline that fetches stereo intertial VSLAM dataset from Google Cloud Storage or local file system"""        
    
    # Demux stereo inertial dataset into three data pipes: Imu0, Cam0, Cam1
    if uri.startswith("gs://"):
        imu_dp, cam0_dp, cam1_dp = IterableWrapper([uri]) \
            .open_files_by_fsspec(mode="rb") \
            .load_from_bz2() \
            .load_from_tar() \
            .demux(3, classifier_fn=classifier, drop_none=True, buffer_size=3000)
    elif uri.endswith(".tar.bz2") or uri.endswith(".tbz"):
        imu_dp, cam0_dp, cam1_dp = IterableWrapper([uri]) \
            .open_files(mode="rb") \
            .load_from_bz2() \
            .load_from_tar() \
            .demux(3, classifier_fn=classifier, drop_none=True, buffer_size=3000)
    elif uri.endswith(".tar"):
        imu_dp, cam0_dp, cam1_dp = IterableWrapper([uri]) \
            .open_files(mode="rb") \
            .load_from_tar() \
            .demux(3, classifier_fn=classifier, drop_none=True, buffer_size=3000)
    else:
        raise Error("unsupoprted uri type: %s" % uri)

    imu_dp = imu_dp \
        .parse_csv(skip_lines=1) \
        .map(lambda x: np.asarray(x).astype(np.double))
    
    # Sort by timestamp in asc order
    cam0_dp = cam0_dp.map(collate_cam)
    cam0_dp = IterableWrapper(sorted(list(cam0_dp), key=itemgetter(0)))    
    cam1_dp = cam1_dp.map(collate_cam)                 
        
    # Collate final data pipe
    dp = cam0_dp.zip_with_iter(cam1_dp, 
                               key_fn=itemgetter(0),
                               ref_key_fn=itemgetter(0),
                               merge_fn=Compounder(imu_dp),
                               keep_key=False)
    
    return dp

## Helper functions

In [19]:
# Start Stereo VSLAM
def track_stereo(slam, dp):
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    t0 = 0
    for t, im0, im1, imu in tqdm(list(dp)):
        if t0 == 0:
            t0 = t
        if len(imu.shape) > 1:
            imu[:,[0]] -= t0
        t = (t-t0) / 1e9
        tcw = slam.track_stereo(clahe.apply(im0), clahe.apply(im1), t, imu)
        yield np.concatenate(([t], tcw))

In [5]:
# Plot local pose estimation
def plot_pose(camera_pose):
    # Transform camera reference frame (X right, Y down, Z forward) to PX4 body reference frame (FRD)
    pose = camera_pose[:,[0,3,1,2,6,4,5,7]]

    fig = figure(width=500, height=500, title="Vision Odometry Position", tooltips=TOOLTIPS)
    fig.line(pose[:,0], pose[:,1], line_width=2, line_color="red", legend_label="X")
    fig.line(pose[:,0], pose[:,2], line_width=2, line_color="green", legend_label="Y")
    fig.line(pose[:,0], pose[:,3], line_width=2, line_color="blue", legend_label="Z")
    fig.legend.location = "top_left"
    fig.xaxis.axis_label = "Time (s)"
    fig.yaxis.axis_label = '[m]'

    # Plot rotation
    rot = Rotation.from_quat(camera_pose[:,[4,5,6,7]]).as_euler('xyz', degrees=True)
    fig2 = figure(width=500, height=500, title="Rotation", tooltips=TOOLTIPS)
    fig2.line(pose[:,0], rot[:,0], line_width=2, line_color="red", legend_label="Roll")
    fig2.line(pose[:,0], rot[:,1], line_width=2, line_color="green", legend_label="Pitch")
    fig2.line(pose[:,0], rot[:,2], line_width=2, line_color="blue", legend_label="Yaw")
    fig2.legend.location = "top_left"
    fig2.xaxis.axis_label = "Time (s)"
    fig2.yaxis.axis_label = "[deg]"

    show(row(fig, fig2))
    
# Plot X-Y map
def plot_xy(camera_pose, width = 400, height = 400):
    # Transform camera reference frame (X right, Y down, Z forward) to PX4 body reference frame (FRD)
    pose = camera_pose[:,[0,3,1,2,6,4,5,7]] 
    p = figure(width=width, height=height, title="XY Map")
    p.circle(pose[:,1], pose[:,2], line_width=1)
    p.xaxis.axis_label = 'X [m]'
    p.yaxis.axis_label = 'Y [m]'

    show(p)    

In [6]:
ORB_VOC = str(Path("../Vocabulary/ORBvoc.txt").resolve())
ORB_CONF = str(Path("datasets/orb-slam3-unity.yaml").resolve())

## Vision Inertial MAP estimation with stereo camera

### Rotate 360° at 15deg/s

In [17]:
dp = VIDataPipe('datasets/sim_rotate_360_15deg.tar')

In [ ]:
# Make a video from loaded dataset
display_video2(dp, (400, 300))

In [ ]:
# Initialize ORB-SLAM3 System
slam = orb_slam3.System(ORB_VOC, ORB_CONF, orb_slam3.Sensor.IMU_STEREO, False, 0)

In [20]:
# Fuse stereo images and run tracking
pose = np.array([x for x in track_stereo(slam, dp)])

  0%|          | 0/529 [00:00<?, ?it/s]

RuntimeError: IMU measurement must have 7 cols

In [ ]:
# Plot VSLAM pose estimation result
plot_pose(pose)

In [ ]:
# Plot X-Y map
plot_xy(pose)

## Vision only MAP estimation with stereo camera

These examples show how to fuse stereo images into ORB-SLAM3 and then get a world pose.

### Evaluate sim-takeoff-20m

Evaluate simulator dataset that ascends to 20m AGL and then land at the same spot.

In [7]:
# Load simulator dataset that ascends to 20m AGL and then land
dp = VIDataPipe("datasets/sim_takeoff_20m.tar")

/usr/src/ORB_SLAM3/python/venv/lib/python3.8/site-packages/torch/utils/data/datapipes/utils/common.py:135: UserWarning: Local function is not supported by pickle, please use regular python function or functools.partial instead.
  warnings.warn(


In [ ]:
# Display video
display_video2(dp, (400,300))    

In [8]:
# Initialize ORB-SLAM3 System
slam = orb_slam3.System(ORB_VOC, ORB_CONF, orb_slam3.Sensor.STEREO, False, 0)


ORB-SLAM3 Copyright (C) 2017-2020 Carlos Campos, Richard Elvira, Juan J. Gómez, José M.M. Montiel and Juan D. Tardós, University of Zaragoza.
ORB-SLAM2 Copyright (C) 2014-2016 Raúl Mur-Artal, José M.M. Montiel and Juan D. Tardós, University of Zaragoza.
This program comes with ABSOLUTELY NO WARRANTY;
This is free software, and you are welcome to redistribute it
under certain conditions. See LICENSE.txt.

Input sensor was set to: Stereo
Loading settings from /usr/src/ORB_SLAM3/python/datasets/orb-slam3-unity.yaml
	-Loaded camera 1
	-Loaded camera 2
	-Loaded image info
	-Loaded ORB settings
	-Loaded viewer settings
	-Loaded Atlas settings
	-Loaded misc parameters
	-Computed rectification maps
----------------------------------
SLAM settings: 
	-Camera 1 parameters (Pinhole): [ 971.159 974.21 399.568 299.672 ]
	-Camera 1 distortion parameters: [  -0.00169682 -0.0052296 0.000343946 -0.000117598 ]
	-Camera 2 parameters (Pinhole: [ 971.235 974.272 399.873 299.429 ]
	-Camera 1 distortion par

Camera1.k3 optional parameter does not exist...
Camera2.k3 optional parameter does not exist...
Camera.newHeight optional parameter does not exist...
Camera.newWidth optional parameter does not exist...
System.LoadAtlasFromFile optional parameter does not exist...
System.SaveAtlasToFile optional parameter does not exist...
System.thFarPoints optional parameter does not exist...


In [9]:
# Fuse stereo images and run tracking
pose = np.array([x for x in track_stereo(slam, dp)])

  0%|          | 0/628 [00:00<?, ?it/s]

First KF:0; Map init KF:0
New Map created with 466 points
TRACK_REF_KF: Less than 15 matches!!
Fail to track local map!
LM: Active map reset recieved
LM: Active map reset, waiting...
LM: Reseting current map in Local Mapping...
LM: End reseting Local Mapping...
LM: Reset free the mutex
LM: Active map reset, Done!!!
mnFirstFrameId = 0
mnInitialFrameId = 0
1 Frames set to lost
First KF:1; Map init KF:0
New Map created with 473 points
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track local map!
Fail to track 

In [10]:
# Plot VSLAM pose estimation result
plot_pose(pose)

In [11]:
# Plot X-Y map
plot_xy(pose)

### Rotate 360 deg in 15deg/s

Evaluate simulator dataset that a vehicle takes off, ascends to 5m AGL, roate 360 deg twice, then land at same spot. Rotation speed is _15deg/s_.

In [12]:
# Load simulator dataset that climbs to 5m AGL and rotate 360deg in 15deg/s
dp = VIDataPipe("datasets/sim_rotate_360_15deg.tar")

In [ ]:
# Display video
display_video2(dp, (400,300))    

In [13]:
# Initialize ORB-SLAM3 System
slam = orb_slam3.System(ORB_VOC, ORB_CONF, orb_slam3.Sensor.STEREO, False, 0)


ORB-SLAM3 Copyright (C) 2017-2020 Carlos Campos, Richard Elvira, Juan J. Gómez, José M.M. Montiel and Juan D. Tardós, University of Zaragoza.
ORB-SLAM2 Copyright (C) 2014-2016 Raúl Mur-Artal, José M.M. Montiel and Juan D. Tardós, University of Zaragoza.
This program comes with ABSOLUTELY NO WARRANTY;
This is free software, and you are welcome to redistribute it
under certain conditions. See LICENSE.txt.

Input sensor was set to: Stereo
Loading settings from /usr/src/ORB_SLAM3/python/datasets/orb-slam3-unity.yaml
	-Loaded camera 1
	-Loaded camera 2
	-Loaded image info
	-Loaded ORB settings
	-Loaded viewer settings
	-Loaded Atlas settings
	-Loaded misc parameters
	-Computed rectification maps
----------------------------------
SLAM settings: 
	-Camera 1 parameters (Pinhole): [ 971.159 974.21 399.568 299.672 ]
	-Camera 1 distortion parameters: [  -0.00169682 -0.0052296 0.000343946 -0.000117598 ]
	-Camera 2 parameters (Pinhole: [ 971.235 974.272 399.873 299.429 ]
	-Camera 1 distortion par

Camera1.k3 optional parameter does not exist...
Camera2.k3 optional parameter does not exist...
Camera.newHeight optional parameter does not exist...
Camera.newWidth optional parameter does not exist...
System.LoadAtlasFromFile optional parameter does not exist...
System.SaveAtlasToFile optional parameter does not exist...
System.thFarPoints optional parameter does not exist...


In [14]:
# Fuse stereo images and run tracking
pose = np.array([x for x in track_stereo(slam, dp)])

  0%|          | 0/529 [00:00<?, ?it/s]

First KF:63; Map init KF:0
New Map created with 443 points
*Loop detected
Local Mapping STOP
Local Mapping RELEASE
Fail to track local map!
Relocalized!!
Local Mapping STOP
Local Mapping RELEASE


In [15]:
# Plot VSLAM pose estimation result
plot_pose(pose)

In [16]:
# Plot X-Y map
plot_xy(pose)

### Sellen Warehouse dataset

In [ ]:
# Load Sellen Warehouse Dataset
dp = VIDataPipe("datasets/sellen-20230126081954.tar")

In [ ]:
# Display video
display_video2(dp, (400,300))

In [ ]:
# Initialize ORB-SLAM3 System
slam = orb_slam3.System(ORB_VOC, ORB_CONF, orb_slam3.Sensor.STEREO, False, 0)

In [ ]:
# Fuse stereo images and run tracking
pose = np.array([x for x in track_stereo2(slam, dp)])

In [ ]:
# Plot VSLAM pose estimation result
plot_pose(pose)

In [ ]:
# Plot X-Y map
plot_xy(pose)

### Dataset sim-circle

In [ ]:
# Load dataset
dp = VIDataPipe("datasets/sim_circle.tar")

In [ ]:
# Make a video from loaded dataset
display_video2(dp, (400, 300))

In [ ]:
# Initialize ORB-SLAM3 System
slam = orb_slam3.System(ORB_VOC, ORB_CONF, orb_slam3.Sensor.STEREO, False, 0)

In [ ]:
# Fuse stereo images and run tracking
pose = np.array([x for x in track_stereo(slam, dp)])

In [ ]:
# Plot VSLAM pose estimation result
plot_pose(pose)

In [ ]:
# Plot X-Y map
plot_xy(pose)